In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
# 노이즈의 크기로 랜덤한 노이즈 입력하고 그 노이즈에서 손글씨 이미지를 무작위로 생성해내도록 할 변수
n_noise = 128
# 노이즈에 힌트를 주기위한 텐서
n_class = 10

# X는 실제이미지를 넣을 변수 Z는 노이즈에서 생성된 가짜이미지를 넣을 변수
# 노이즈에 레이블 데이터를 힌트로 넣어주기 위해 Y텐서 선언
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

###tf.layers 사용 = 각각 다른 변수의 선언 필요없이 스코프를 지정해 각 스코프에 해당하는 변수들만 따로 불러 올 수 있습니다.
#생성자 신경망에 사용할 변수 설정
#G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
#G_b1 = tf.Variable(tf.zeros([n_hidden]))
#G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
#G_b2 = tf.Variable(tf.zeros([n_input]))

#구분자 신경망에 사용할 변수 설정
#D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
#D_b1 = tf.Variable(tf.zeros([n_hidden]))
#D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
#D_b2 = tf.Variable(tf.zeros([1]))

# 생성자, 구분자 신경망 구성
# 생성자신경망
## tf.concat() => noise 값에 labels정보(힌트)를 추가해줌.
## tf.layers.dense() => 은닉층을 만들어주고, 진짜 이미지와 같은 크기의 값을 만드는 출력층을 만들어줌.
def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 추가합니다.
        inputs = tf.concat([noise, labels], 1)

        # TensorFlow 에서 제공하는 유틸리티 함수를 이용해 신경망을 매우 간단하게 구성할 수 있습니다.
        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,
                                 activation=tf.nn.sigmoid)

    return output

                   
# 구분자신경망
# 진짜이미지를 판별할 때 가짜이미지를 판별할 때 똑같은 변수를 사용해야 한다는 것
## scope.reuse_variabels 함수를 이용해 이전에 사용한 변수를 재사용하도록 작성.
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기 위해,
        # 이전에 사용되었던 변수를 재사용하도록 합니다.
        if reuse:
            scope.reuse_variables()

        inputs = tf.concat([inputs, labels], 1)

        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1,
                                 activation=None)

    return output
                   
## 노이즈 생성해주는 함수 이전과는 다르게 균등분포로 생성.
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

# 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만듦
# G가 만든 가짜 이미지와 진짜 이미지를 각각 구분자에 넣어 입력한 이미지가 진짜 인지 판별 하도록 함.
## 힌트 Y를 추가해줌
G = generator(Z, Y)
## True는 가짜이미지인것을 알려주기위한 플래그. 즉 변수들을 재사용 하도록 알려주는것.
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# 손실값
# 두개가 필요함. 경찰 학습용 손실값과, 위조지폐범 학습용을 구함.
# 경찰 학습용 : 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값
# D_gene는 가짜를 판별하는 값이므로 0에 가까워야 함.(가짜라고 판별)
# D_real은 진짜 이미지 판별값이므로 1에 가까워야 함.(진짜라고 판별)
#loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

## 이전보다 조금더 간편하게 작성
## tf.ones_like() => D_real의 크기만큼 1로 채운 값들과 D_real의 결과 값을 비교하는 함수.
## tf.zeros_like() => D_gene의 크기만큼 0으로 채운 값들과 D_gene 의 결과 값을 비교하는 함수.
loss_D_real = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.zeros_like(D_gene)))

loss_D = loss_D_real + loss_D_gene


# 범인 학습용 : 생성자가 만든 이미지를 구분자가 진짜라고 판단 하도록 하는 손실값.
# D_gene를 1에 가깝게 만들기만 하면됨.
# 즉 가짜 이미지를 넣어도 진짜같다고 판별해야 함. D_gene값을 최대화 하면 위조지폐범을 학습 시킬 수 있음.
#loss_G = tf.reduce_mean(tf.log(D_gene))
## 조금 다르게 작성
loss_G = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_gene, labels=tf.ones_like(D_gene)))

# 결과적으로 GAN의 목표는 loss_D와 loss_G를 모두 최대화 하는 것
# 하지만 서로 반비례 관계(경쟁관계)에 있기때문에 항상 같이 증가하는 경향을 보이지는 않음.

# 손실값을 이용해 학습
# loss_D를 구할 때는 구분자 신경망에 사용되는 변수들만 사용.
# loss_G를 구할 때는 생성자 신경망에 사용되는 변수들만 사용.
# 그래야 각각 학습 시켰을때 신경망의 변화가 생기지 않는다.
#D_var_list = [D_W1, D_b1, D_W2, D_b2]
#G_var_list = [G_W1, G_b1, G_W2, G_b2]

# 최적화 함수 구성
# GAN의 목표는 loss_D, loss_G의 값이 최대값이 되는 것.
# 최적화함수는 minimize뿐이므로 "-" 를 붙여줌.
#train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
#train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)
## tf.get_collection 함수를 이용해 discriminator와 generator 스코프에서 사용된 변수들을 가져옴
## 각각의 최적화 함수에 넣어 최적화 시킴
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                           scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,
                                            var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,
                                            var_list=vars_G)

##########
#학습 코드
##########
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0


for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        ## 힌트를 주기위해 Y값을 넣어줌.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Y: batch_ys, Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    
    ##############
    #학습결과 확인
    ##############
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,
                           feed_dict={Y: mnist.test.labels[:sample_size],
                                      Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

    

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0000 D loss: 0.00934 G loss: 7.827
Epoch: 0001 D loss: 0.002937 G loss: 9.528
Epoch: 0002 D loss: 0.0522 G loss: 8.519
Epoch: 0003 D loss: 0.00488 G loss: 7.85
Epoch: 0004 D loss: 0.01589 G loss: 7.421
Epoch: 0005 D loss: 0.01609 G loss: 8.33
Epoch: 0006 D loss: 0.02639 G loss: 7.895
Epoch: 0007 D loss: 0.02975 G loss: 8.713
Epoch: 0008 D loss: 0.0377 G loss: 7.526
Epoch: 0009 D loss: 0.0181 G loss: 7.656
Epoch: 0010 D loss: 0.1229 G loss: 6.475
Epoch: 0011 D loss: 0.2131 G loss: 4.736
Epoch: 0012 D loss: 0.2158 G loss: 6.446
Epoch: 0013 D loss: 0.1977 G loss: 4.968
Epoch: 0014 D loss: 0.3443 G loss: 5.239
Epoch: 0015 D loss: 0.3813 G loss: 4.388
Epoch: 0016 D loss: 0.3103 G loss: 4.198
Epoch: 0017 D loss: 0.3205 G loss: 4.126
Epoch: 0018 D loss: 0.4484 G loss: 3.96
Epoch: 0019 D 